In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import logging
import shutil
from omegaconf import OmegaConf
from hydra import initialize, compose
from hydra.core.config_store import ConfigStore
import mlflow
from src.experiments.sner import sner
from pathlib import Path

import os

logging.basicConfig(
    format="%(asctime)s %(levelname)s:%(message)s",
    level=logging.INFO,
    datefmt="%I:%M:%S",
)
logger = logging.getLogger("training")

In [ ]:
mlflow.get_tracking_uri()

In [4]:
sentence_length = 106

run_experiments = os.getenv("UVL_BERT_RUN_EXPERIMENTS", "True") == "True"
pin_commits = os.getenv("UVL_BERT_PIN_COMMITS", "True") == "FALSE"

print(f"{run_experiments=}")
print(f"{pin_commits=}")

run_experiments=True
pin_commits=False


In [5]:
from tooling.config import Experiment, Transformation

smote_experiment_config = Experiment(
    name="SMOTE_BERT_Config",
    iterations=5,
    force=False,
    dataset="all",
    lower_case=True,
    smote = True,
    smote_k_neighbors = 5,
    smote_sampling_strategy = 'not majority'
)

levels_transformation_config = Transformation(
    description="Levels",
    type="Reduced",
    task="Domain_Level",
    domain_data="Domain_Level",
    activity="Domain_Level",
    stakeholder="Domain_Level",
    system_function="Interaction_Level",
    interaction="Interaction_Level",
    interaction_data="Domain_Level",
    workspace="Interaction_Level",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

label_transformation_config = Transformation(
    description="None",
    type="Full",
    task="Task",
    domain_data="Domain_Data",
    activity="Activity",
    stakeholder="Stakeholder",
    system_function="System_Function",
    interaction="Interaction",
    interaction_data="Interaction_Data",
    workspace="Workspace",
    software="System_Level",
    internal_action="System_Level",
    internal_data="System_Level",
    system_level="System_Level",
)

In [6]:
from tooling.transformation import get_hint_transformation
import pickle

hint_transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(levels_transformation_config)
)

Path("./src/service/models/").mkdir(parents=True, exist_ok=True)

hint_label2id = hint_transformation["label2id"]
pickle.dump(
    hint_label2id, open("./src/service/models/hint_label2id.pickle", "wb")
)
hint_id2label = {y: x for x, y in hint_label2id.items()}
pickle.dump(
    hint_id2label, open("./src/service/models/hint_id2label.pickle", "wb")
)

transformation = get_hint_transformation(
    transformation_cfg=OmegaConf.structured(label_transformation_config)
)
label2id = transformation["label2id"]
pickle.dump(label2id, open("./src/service/models/label2id.pickle", "wb"))
id2label = {y: x for x, y in label2id.items()}
pickle.dump(id2label, open("./src/service/models/id2label.pickle", "wb"))

08:46:46 INFO:Hint Label2Id: hint_label2id={'0': 0, 'System_Level': 1, 'Domain_Level': 2, 'Interaction_Level': 3}
08:46:46 INFO:Hint Label2Id: hint_label2id={'0': 0, 'Task': 1, 'Domain_Data': 2, 'Activity': 3, 'Stakeholder': 4, 'System_Function': 5, 'Interaction': 6, 'Interaction_Data': 7, 'Workspace': 8, 'System_Level': 9}


## Train BERT First Stage Model with SMOTE


In [7]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_1_experiment_config = deepcopy(smote_experiment_config)
bert_1_experiment_config.name = "SMOTE_BERT_TORE_LEVELS"
bert_1_experiment_config.force = False

bert_1_config = BERT(
    max_len=123,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=True,
    learning_rate_bert=3e-05,
    learning_rate_classifier=0.0005,
    validation_batch_size=64,
)


bert_1_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_1_config,
        experiment=bert_1_experiment_config,
        transformation=levels_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_1_cfg))

bert_1_run_id = get_run_id(bert_1_cfg, pin_commit=pin_commits)

print(bert_1_run_id)

bert_1_run = mlflow.get_run(bert_1_run_id)
mlflow.artifacts.download_artifacts(
    f"{bert_1_run.info.artifact_uri}/0_model",
    dst_path=Path("./src/service/models/"),
)
try:
    shutil.rmtree(Path("./src/service/models/bert_1"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert_1")

08:46:47 INFO:PyTorch version 2.2.2 available.
08:46:47 INFO:TensorFlow version 2.16.1 available.
2024-05-07 20:46:47.984028: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-07 20:46:48.216664: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 20:46:48.814112: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
08:46:49 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 64
  number_epoch

Entered apply_smote()
Counter original:  Counter({'Domain_Level': 5659, 'Interaction_Level': 4253, 'System_Level': 1729})


08:46:52 INFO:Created fold datasets for fold: 0, stored at paths=[PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/exultant-mouse-891/0_data_train.pickle'), PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/exultant-mouse-891/0_data_train.csv'), PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/exultant-mouse-891/0_data_test.pickle'), PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/exultant-mouse-891/0_data_test.csv')]
08:46:52 INFO:Starting iteration=0


Counter SMOTE:  Counter({'System_Level': 5659, 'Interaction_Level': 5659, 'Domain_Level': 5659})


Map:   0%|          | 0/7824 [00:00<?, ? examples/s]

...

09:11:38 INFO:Logged iteration result res.precision=0.7970479707672539 res.recall=0.7901783427143391


{'eval_loss': 1.69013512134552, 'eval_step': 14, 'eval_precision': 0.7970479707672539, 'eval_recall': 0.7901783427143391, 'eval_f1': 0.7935982906220884, 'eval_label_count': 4, 'eval_runtime': 0.6415, 'eval_samples_per_second': 969.643, 'eval_steps_per_second': 15.589, 'epoch': 8.0}
{'train_runtime': 280.5726, 'train_samples_per_second': 220.093, 'train_steps_per_second': 27.515, 'train_loss': 0.14120248661140086, 'epoch': 8.0}


09:11:40 INFO:Logged iteration result res.precision=0.7970479707672539 res.recall=0.7901783427143391
09:11:40 INFO:Logged iteration result res.precision=0.7970479707672539 res.recall=0.7901783427143391
09:11:40 INFO:Finished iteration=4
09:11:40 INFO:Logging model artifact (might take a while)
09:11:41 INFO:Breaking early after iteration=4 of 5 folds
09:11:42 INFO:Logged experiment result res.mean_precision=0.8022905809721879 res.mean_recall=0.7859404833318002
09:11:42 INFO:Left mlflow context


dbff7bc6f18f41039ac34eebd0c5824f


PosixPath('src/service/models/bert_1')

## Train BERT E2E Model with SMOTE


In [8]:
from tooling.observability import get_run_id
from tooling.config import BERTConfig, BERT

from copy import deepcopy

bert_experiment_config = deepcopy(smote_experiment_config)
bert_experiment_config.name = "SMOTE_BERT_TORE_CATEGORIES"

bert_config = BERT(
    learning_rate_bert=6e-05,
    learning_rate_classifier=0.1,
    max_len=123,
    number_epochs=8,
    train_batch_size=8,
    weight_decay=0.01,
    weighted_classes=False,
)

bert_cfg = OmegaConf.structured(
    BERTConfig(
        bert=bert_config,
        experiment=bert_experiment_config,
        transformation=label_transformation_config,
    )
)

if run_experiments:
    from experiments.bert import bert

    bert(OmegaConf.create(bert_cfg))

bert_run_id = get_run_id(bert_cfg, pin_commit=pin_commits)

print(bert_run_id)

run = mlflow.get_run(bert_run_id)
mlflow.artifacts.download_artifacts(
    f"{run.info.artifact_uri}/0_model", dst_path=Path("./src/service/models/")
)
try:
    shutil.rmtree(Path("./src/service/models/bert"))
except FileNotFoundError:
    pass
Path("./src/service/models/0_model").rename("./src/service/models/bert")

09:11:49 INFO:
bert:
  model: bert-base-uncased
  type: BERT
  max_len: 123
  train_batch_size: 8
  validation_batch_size: 32
  number_epochs: 8
  learning_rate_bert: 6.0e-05
  learning_rate_classifier: 0.1
  weight_decay: 0.01
  weighted_classes: false
experiment:
  name: SMOTE_BERT_TORE_CATEGORIES
  description: ''
  random_state: 125
  folds: 5
  iterations: 5
  average: macro
  dataset: all
  lower_case: true
  force: false
  pin_commit: false
  smote: true
  smote_k_neighbors: 5
  smote_sampling_strategy: not majority
transformation:
  description: None
  type: Full
  task: Task
  goals: null
  domain_data: Domain_Data
  activity: Activity
  stakeholder: Stakeholder
  system_function: System_Function
  interaction: Interaction
  interaction_data: Interaction_Data
  workspace: Workspace
  software: System_Level
  internal_action: System_Level
  internal_data: System_Level
  system_level: System_Level

09:11:49 INFO:New experiment. Running
09:11:49 INFO:Entering mlflow context
09:11

Entered apply_smote()
Counter original:  Counter({'Interaction_Data': 2437, 'Interaction': 2302, 'Domain_Data': 1772, 'System_Level': 1729, 'System_Function': 1437, 'Task': 527, 'Workspace': 514, 'Activity': 469, 'Stakeholder': 454})


09:11:52 INFO:Created fold datasets for fold: 0, stored at paths=[PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/youthful-turtle-113/0_data_train.pickle'), PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/youthful-turtle-113/0_data_train.csv'), PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/youthful-turtle-113/0_data_test.pickle'), PosixPath('/home/daniel/uvl-tore-classifier-bert/src/data/temp/sampling/youthful-turtle-113/0_data_test.csv')]
09:11:52 INFO:Starting iteration=0


Counter SMOTE:  Counter({'System_Level': 2437, 'Interaction': 2437, 'Interaction_Data': 2437, 'Workspace': 2437, 'Domain_Data': 2437, 'Activity': 2437, 'Stakeholder': 2437, 'Task': 2437, 'System_Function': 2437})


Map:   0%|          | 0/12780 [00:00<?, ? examples/s]

...

09:48:59 INFO:Logged iteration result res.precision=0.6409607265147258 res.recall=0.6293186025853903


{'eval_loss': 1.6402390003204346, 'eval_step': 14, 'eval_precision': 0.6409607265147258, 'eval_recall': 0.6293186025853903, 'eval_f1': 0.6350863144535552, 'eval_label_count': 10, 'eval_runtime': 0.6632, 'eval_samples_per_second': 937.927, 'eval_steps_per_second': 30.158, 'epoch': 8.0}
{'train_runtime': 434.4988, 'train_samples_per_second': 224.424, 'train_steps_per_second': 28.06, 'train_loss': 0.41525539622844987, 'epoch': 8.0}


09:49:02 INFO:Logged iteration result res.precision=0.6409607265147258 res.recall=0.6293186025853903
09:49:02 INFO:Logged iteration result res.precision=0.6409607265147258 res.recall=0.6293186025853903
09:49:02 INFO:Finished iteration=4
09:49:02 INFO:Logging model artifact (might take a while)
09:49:03 INFO:Breaking early after iteration=4 of 5 folds
09:49:04 INFO:Logged experiment result res.mean_precision=0.6526735799491488 res.mean_recall=0.6264227256225938
09:49:04 INFO:Left mlflow context


1129e9939932491e899979110c3e45d6


PosixPath('src/service/models/bert')